## Create a Cuda code file

In [11]:
%%writefile L7.cu

#include <stdio.h>
#include <cuda_runtime.h>

#define N 4  // Reduced size for easy debugging; set to 1024 for larger matrices

// CUDA Kernel for Matrix Transposition using shared memory
__global__ void transpose(float *in, float *out, int width) {
    __shared__ float tile[32][32 + 1];  // Shared memory with padding to avoid bank conflicts

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int tidx = threadIdx.x;
    int tidy = threadIdx.y;

    if (x < width && y < width) {
        tile[tidy][tidx] = in[y * width + x];
    }
    __syncthreads();

    x = blockIdx.y * blockDim.y + threadIdx.x;
    y = blockIdx.x * blockDim.x + threadIdx.y;

    if (x < width && y < width) {
        out[y * width + x] = tile[tidx][tidy];
    }
}

// CPU Transposition Function
void cpu_transpose(float *in, float *out, int width) {
    for (int i = 0; i < width; ++i) {
        for (int j = 0; j < width; ++j) {
            out[j * width + i] = in[i * width + j];
        }
    }
}

// Print Matrix
void print_matrix(float *matrix, int width) {
    for (int i = 0; i < width; ++i) {
        for (int j = 0; j < width; ++j) {
            printf("%6.1f ", matrix[i * width + j]);
        }
        printf("\n");
    }
    printf("\n");
}

// Verify Result
void verify_result(float *cpu_result, float *gpu_result, int width) {
    for (int i = 0; i < width; ++i) {
        for (int j = 0; j < width; ++j) {
            if (cpu_result[i * width + j] != gpu_result[i * width + j]) {
                printf("Mismatch at (%d, %d): CPU=%f, GPU=%f\n",
                       i, j, cpu_result[i * width + j], gpu_result[i * width + j]);
                return;
            }
        }
    }
    printf("Verification Passed!\n");
}

int main() {
    int size = N * N * sizeof(float);
    float *h_in, *h_out, *h_ref;
    float *d_in, *d_out;

    // Allocate host memory
    h_in = (float *)malloc(size);
    h_out = (float *)malloc(size);
    h_ref = (float *)malloc(size);

    // Initialize input matrix
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            h_in[i * N + j] = i * N + j + 1;  // Fill matrix with 1, 2, ..., N^2
        }
    }

    // Transpose on CPU
    cpu_transpose(h_in, h_ref, N);

    // Allocate device memory
    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);

    // Copy data from host to device
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);

    // Define block and grid dimensions
    dim3 blockDim(32, 32);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);

    // Launch kernel
    transpose<<<gridDim, blockDim>>>(d_in, d_out, N);

    // Copy result back to host
    cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);

    // Print matrices
    printf("Matrix A (Input):\n");
    print_matrix(h_in, N);

    printf("Matrix A Transposed by GPU:\n");
    print_matrix(h_out, N);

    printf("Matrix A Transposed by CPU:\n");
    print_matrix(h_ref, N);

    // Verify results
    verify_result(h_ref, h_out, N);

    // Free memory
    cudaFree(d_in);
    cudaFree(d_out);
    free(h_in);
    free(h_out);
    free(h_ref);

    return 0;
}


Overwriting L7.cu


## Check Driver versions

In [7]:
!gcc --version
!nvcc --version
!nvidia-smi

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Wed Nov 20 19:16:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      

## Compile using Cuda Compiler and Run the File

In [12]:
!nvcc L7.cu -o L7
!./L7

Matrix A (Input):
   1.0    2.0    3.0    4.0 
   5.0    6.0    7.0    8.0 
   9.0   10.0   11.0   12.0 
  13.0   14.0   15.0   16.0 

Matrix A Transposed by GPU:
   1.0    5.0    9.0   13.0 
   2.0    6.0   10.0   14.0 
   3.0    7.0   11.0   15.0 
   4.0    8.0   12.0   16.0 

Matrix A Transposed by CPU:
   1.0    5.0    9.0   13.0 
   2.0    6.0   10.0   14.0 
   3.0    7.0   11.0   15.0 
   4.0    8.0   12.0   16.0 

Verification Passed!
